### Group redirection. Merging dataframe of ICO features ONLY. Merging 1-Hot encoding columns from Mina with success_df.

In [1]:
import pandas as pd
from path import Path

import requests
import pprint
import requests
from requests.exceptions import HTTPError
from datetime import datetime as dt
import pandas as pd
import numpy as np
import timeit
import time
import datetime
import copy

import numpy as np
from sklearn import preprocessing
%matplotlib inline

In [31]:
## Call christians tweets
twitter_df = pd.read_csv(Path('../../../../data/cleandata/twitter_scores.csv'))

In [32]:
twitter_df.head(2)

,Unnamed: 0,Compound,Negative,Neutral,Positive
0,Ethereum,0.147122,0.01278,0.9201,0.06712
1,Binance Coin,0.081454,0.00598,0.9694,0.02460


In [33]:
### rename netural, positive, negative to tweeter_neutral, tweeter_positive, tweeter_negative
twitter_df.rename(columns ={'Unnamed: 0':'Name','Negative':'Twitter_Negative', 'Positive':'Twitter_Positive', 'Neutral':'Twitter_Neutral', 'Compound':'Twitter_Compound'}, inplace = True)

In [34]:
twitter_df.head(2)

,Name,Twitter_Compound,Twitter_Negative,Twitter_Neutral,Twitter_Positive
0,Ethereum,0.147122,0.01278,0.9201,0.06712
1,Binance Coin,0.081454,0.00598,0.9694,0.02460


In [35]:
twitter_df.columns

Index(['Name', 'Twitter_Compound', 'Twitter_Negative', 'Twitter_Neutral',
       'Twitter_Positive'],
      dtype='object')

In [36]:
### append to success_df and one_hot_features

In [37]:
success_df = pd.read_csv(Path(f'../../../../data/cleandata/merges/success_df.csv'))

In [38]:
success_df.head(2)

,rank_cnc,Name,Ticker,market_cap,price,volume24,pct1h,pct24h,pct7d,country,...,pre_Duration,compound,negative,neutral,positive,no_of_posts,Longevity,custom_index,custom_index_scaled,success
0,2,Ethereum,ETH,"20,155,411,771",185.67,"7,906,136,399",-0.10,-1.20,-0.96,Switzerland,...,0 days 00:00:00.000000000,0.460570,0.06116,0.02739,0.90151,100,779,144636.93,0.129162,1
1,7,Binance Coin,BNB,"3,277,372,658",21.07,"231,846,713",-0.46,-1.33,2.49,Japan,...,0 days 00:00:00.000000000,0.563773,0.09667,0.04141,0.86189,100,848,17867.36,0.015956,1


In [39]:
success_df.shape

(573, 29)

In [40]:
success_df.columns

Index(['rank_cnc', 'Name', 'Ticker', 'market_cap', 'price', 'volume24',
       'pct1h', 'pct24h', 'pct7d', 'country', 'platform', 'pre_ico_end',
       'pre_ico_start', 'rating', 'status', 'type', 'End', 'Start', 'Duration',
       'pre_Duration', 'compound', 'negative', 'neutral', 'positive',
       'no_of_posts', 'Longevity', 'custom_index', 'custom_index_scaled',
       'success'],
      dtype='object')

In [41]:
success_df.drop(columns = ['rank_cnc', 'market_cap', 'price', 'volume24',
       'pct1h', 'pct24h', 'pct7d', 'country', 'platform', 'pre_ico_end',
       'pre_ico_start', 'rating', 'status', 'type', 'End', 'Start', 'custom_index', 'custom_index_scaled'], inplace = True)

In [42]:
success_df.columns

Index(['Name', 'Ticker', 'Duration', 'pre_Duration', 'compound', 'negative',
       'neutral', 'positive', 'no_of_posts', 'Longevity', 'success'],
      dtype='object')

In [43]:
success_df.head(2)

,Name,Ticker,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,Longevity,success
0,Ethereum,ETH,0 days 00:00:00.000000000,0 days 00:00:00.000000000,0.460570,0.06116,0.02739,0.90151,100,779,1
1,Binance Coin,BNB,20 days 00:00:00.000000000,0 days 00:00:00.000000000,0.563773,0.09667,0.04141,0.86189,100,848,1


In [44]:
success_df.dtypes

Name             object
Ticker           object
Duration         object
pre_Duration     object
compound        float64
negative        float64
neutral         float64
positive        float64
no_of_posts       int64
Longevity         int64
success           int64
dtype: object

In [45]:
for i in range(len(success_df)) : 
    success_df.loc[i, 'Duration'] = int(success_df.loc[i, 'Duration'][:-24])

In [46]:
for i in range(len(success_df)) : 
    success_df.loc[i, 'pre_Duration'] = int(success_df.loc[i, 'pre_Duration'][:-24])

In [47]:
success_df.head()

,Name,Ticker,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,Longevity,success
0,Ethereum,ETH,0,0,0.460570,0.061160,0.027390,0.901510,100,779,1
1,Binance Coin,BNB,20,0,0.563773,0.096670,0.041410,0.861890,100,848,1
2,EOS,EOS,350,0,0.828470,0.032200,0.858550,0.109270,100,522,0
3,Cardano,ADA,488,0,0.589474,0.098387,0.032946,0.868753,93,1049,0
4,Huobi Token,HT,0,0,0.525717,0.081630,0.033480,0.884940,100,779,1


In [48]:
success_df.dtypes

Name             object
Ticker           object
Duration          int64
pre_Duration      int64
compound        float64
negative        float64
neutral         float64
positive        float64
no_of_posts       int64
Longevity         int64
success           int64
dtype: object

In [49]:
one_hot_features = pd.read_csv(Path(f"../../../../data/cleandata/encoded.csv"))

In [50]:
one_hot_features.head(2)

,Ticker,country_Argentina,country_Armenia,country_Australia,country_Austria,country_Azerbaijan,country_Bahamas,country_Belarus,country_Belgium,country_Belize,...,platform_Hyperledger,platform_Litecoin,platform_NEM,platform_NEO,platform_Nxt,platform_Omni,platform_Own Blockchain,platform_Scrypt,platform_Stellar,platform_Waves
0,ETH,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BNB,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
one_hot_features.shape

(573, 84)

In [52]:
#merging twitter and success by Name
merged = pd.merge(left=success_df, right=twitter_df, left_on='Name', right_on='Name')

In [53]:
merged.head(2)

,Name,Ticker,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,Longevity,success,Twitter_Compound,Twitter_Negative,Twitter_Neutral,Twitter_Positive
0,Ethereum,ETH,0,0,0.460570,0.06116,0.02739,0.90151,100,779,1,0.147122,0.01278,0.9201,0.06712
1,Binance Coin,BNB,20,0,0.563773,0.09667,0.04141,0.86189,100,848,1,0.081454,0.00598,0.9694,0.02460


In [54]:
merged.shape

(573, 15)

In [55]:
merged_final = pd.merge(left=merged, right=one_hot_features, left_on='Ticker', right_on='Ticker')

In [56]:
merged_final.shape

(573, 98)

In [57]:
merged_final.head(2)

,Name,Ticker,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,Longevity,...,platform_Hyperledger,platform_Litecoin,platform_NEM,platform_NEO,platform_Nxt,platform_Omni,platform_Own Blockchain,platform_Scrypt,platform_Stellar,platform_Waves
0,Ethereum,ETH,0,0,0.460570,0.06116,0.02739,0.90151,100,779,...,0,0,0,0,0,0,0,0,0,0
1,Binance Coin,BNB,20,0,0.563773,0.09667,0.04141,0.86189,100,848,...,0,0,0,0,0,0,0,0,0,0


In [59]:
merged_final.head()

,Name,Ticker,Duration,pre_Duration,compound,negative,neutral,positive,no_of_posts,Longevity,...,platform_Hyperledger,platform_Litecoin,platform_NEM,platform_NEO,platform_Nxt,platform_Omni,platform_Own Blockchain,platform_Scrypt,platform_Stellar,platform_Waves
0,Ethereum,ETH,0,0,0.460570,0.061160,0.027390,0.901510,100,779,...,0,0,0,0,0,0,0,0,0,0
1,Binance Coin,BNB,20,0,0.563773,0.096670,0.041410,0.861890,100,848,...,0,0,0,0,0,0,0,0,0,0
2,EOS,EOS,350,0,0.828470,0.032200,0.858550,0.109270,100,522,...,0,0,0,0,0,0,1,0,0,0
3,Cardano,ADA,488,0,0.589474,0.098387,0.032946,0.868753,93,1049,...,0,0,0,0,0,0,1,0,0,0
4,Huobi Token,HT,0,0,0.525717,0.081630,0.033480,0.884940,100,779,...,0,0,0,0,0,0,0,0,0,0


In [61]:
#find ICO at 360 days or more...
merged_360_ICO = merged_final[merged_final['Longevity'] < 360]

In [66]:
merged_360_ICO.shape

In [84]:
np.where(pd.isnull(merged_final))

(array([ 97,  97,  97,  97, 105, 105, 105, 105, 124, 124, 124, 124, 127,
        127, 127, 127, 132, 132, 132, 132, 139, 139, 139, 139, 157, 157,
        157, 157, 175, 175, 175, 175, 180, 180, 180, 180, 187, 187, 187,
        187, 202, 202, 202, 202, 209, 209, 209, 209, 218, 218, 218, 218,
        246, 246, 246, 246, 252, 252, 252, 252, 258, 258, 258, 258, 262,
        262, 262, 262, 265, 265, 265, 265, 271, 271, 271, 271, 296, 296,
        296, 296, 300, 300, 300, 300, 309, 309, 309, 309, 315, 315, 315,
        315, 331, 331, 331, 331, 333, 333, 333, 333, 334, 334, 334, 334,
        337, 337, 337, 337, 339, 339, 339, 339, 345, 345, 345, 345, 349,
        349, 349, 349, 357, 357, 357, 357, 376, 376, 376, 376, 382, 382,
        382, 382, 388, 388, 388, 388, 395, 395, 395, 395, 403, 403, 403,
        403, 411, 411, 411, 411, 413, 413, 413, 413, 422, 422, 422, 422,
        427, 427, 427, 427, 437, 437, 437, 437, 443, 443, 443, 443, 449,
        449, 449, 449, 460, 460, 460, 460, 465, 465

In [85]:
np.where(merged_final.applymap(lambda x: x == ''))

(array([], dtype=int64), array([], dtype=int64))

In [82]:
merged_360_ICO.iloc[27,11]

nan

In [77]:
np.where(pd.isnull(one_hot_features))

(array([], dtype=int64), array([], dtype=int64))

In [76]:
np.where(one_hot_features.applymap(lambda x: x == ''))

(array([], dtype=int64), array([], dtype=int64))

In [78]:
np.where(pd.isnull(twitter_df))

(array([ 97,  97,  97,  97, 105, 105, 105, 105, 124, 124, 124, 124, 127,
        127, 127, 127, 132, 132, 132, 132, 139, 139, 139, 139, 157, 157,
        157, 157, 175, 175, 175, 175, 180, 180, 180, 180, 187, 187, 187,
        187, 202, 202, 202, 202, 209, 209, 209, 209, 218, 218, 218, 218,
        246, 246, 246, 246, 252, 252, 252, 252, 258, 258, 258, 258, 262,
        262, 262, 262, 265, 265, 265, 265, 271, 271, 271, 271, 296, 296,
        296, 296, 300, 300, 300, 300, 309, 309, 309, 309, 315, 315, 315,
        315, 331, 331, 331, 331, 333, 333, 333, 333, 334, 334, 334, 334,
        337, 337, 337, 337, 339, 339, 339, 339, 345, 345, 345, 345, 349,
        349, 349, 349, 357, 357, 357, 357, 376, 376, 376, 376, 382, 382,
        382, 382, 388, 388, 388, 388, 395, 395, 395, 395, 403, 403, 403,
        403, 411, 411, 411, 411, 413, 413, 413, 413, 422, 422, 422, 422,
        427, 427, 427, 427, 437, 437, 437, 437, 443, 443, 443, 443, 449,
        449, 449, 449, 460, 460, 460, 460, 465, 465

In [79]:
np.where(twitter_df.applymap(lambda x: x == ''))

(array([], dtype=int64), array([], dtype=int64))

In [80]:
np.where(pd.isnull(success_df))

(array([], dtype=int64), array([], dtype=int64))

In [81]:
np.where(success_df.applymap(lambda x: x == ''))

In [90]:
#filling nas with 0
merged_final.fillna(0, inplace= True)

In [91]:
#find ICO at 360 days or less...
merged_post_ICO = merged_final[merged_final['Longevity'] >= 360] 
merged_post_ICO.shape

(537, 98)

In [92]:
file_path = Path("../../../../data/cleandata/1hot_merged_post_ICO.csv")
merged_post_ICO.to_csv(file_path, index=False)

In [93]:
file_path = Path("../../../../data/cleandata/1hot_merged_360_ICO.csv")
merged_360_ICO.to_csv(file_path, index=False)